# Asynchonous Operation
DNAC is a collection of stateless microservices.   All of the PUT/POST/DELETE operations are non-blocking (asynchronous).


When an API resquest, the user gets a task that they need to poll until it completes.  The task will contain status information about the result.


### Login
This is a simplified login approach, where a module is used.

In [1]:
from login import login
import json
dnac = login()

https://sandboxdnac.cisco.com/api/system/v1/auth/token


## Pathtrace
The pathtrace service traces the path between two hosts across the network.  The first call to start a pathtrace will return a task.


In [2]:
flowTask = dnac.flowanalysis.initiateFlowAnalysis(flowAnalysisRequest={"sourceIP": '10.10.22.114',
                                                                           "destIP" : '10.10.22.98',
                                                                           "periodicRefresh" : False})

https://sandboxdnac.cisco.com/api/v1/flow-analysis


The response contains a 'taskId' to monitor the status of the request

In [3]:
print (json.dumps(dnac.serialize(flowTask), indent=4))

{
    "version": "1.0",
    "response": {
        "taskId": "e9e9335d-8ff6-4c26-82fa-13ba1d8e812e",
        "url": "/api/v1/flow-analysis/7a672a66-5baa-4f39-ba54-1066f49b876f",
        "flowAnalysisId": "7a672a66-5baa-4f39-ba54-1066f49b876f"
    }
}


### Tasks
The 'wait_for_task_complete' method takes a task and an optional timout and polling_frequency.  It monitors the status of the task, and returns when the task is complete (or the timeout is reached).

In [5]:
# extrace the taskId
taskId = flowTask.response.taskId

# wait_for_task_complete is a utility function that polls APIC-EM until the task is completed.  
# You can provide an optional timeout and polling interval
taskStatus = dnac.task_util.wait_for_task_complete(flowTask, timeout=15, poll_frequency=5)

print(json.dumps(dnac.serialize(taskStatus), indent=4))

https://sandboxdnac.cisco.com/api/v1/task/e9e9335d-8ff6-4c26-82fa-13ba1d8e812e
{
    "rootId": "e9e9335d-8ff6-4c26-82fa-13ba1d8e812e",
    "progress": "7a672a66-5baa-4f39-ba54-1066f49b876f",
    "version": 1516598584241,
    "id": "e9e9335d-8ff6-4c26-82fa-13ba1d8e812e",
    "isError": false,
    "serviceType": "Policy Analysis Service",
    "endTime": 1516598584252,
    "startTime": 1516598583874,
    "lastUpdate": 1516598584241
}


### 'progress' attribute
The 'progress' attribute from the completed task is often used to provide information about the request.  In the case of a POST, it is usually an 'id' of the nely created resource.

In [6]:
# this is the id of the pathtrace result that we need to GET.
flowId = taskStatus.progress

# this will get the pathTrace
pathTrace = dnac.flowanalysis.getFullFlowAnalysisResult(flowAnalysisId=flowId)

print (json.dumps(dnac.serialize(pathTrace.response),indent=4))

https://sandboxdnac.cisco.com/api/v1/flow-analysis/7a672a66-5baa-4f39-ba54-1066f49b876f
{
    "networkElementsInfo": [
        {
            "type": "wired",
            "linkInformationSource": "Wired",
            "id": "ac385e41-165b-4de7-a49b-42ba15cacd4f",
            "ip": "10.10.22.114"
        },
        {
            "ingressInterface": {
                "physicalInterface": {
                    "name": "TenGigabitEthernet1/0/24",
                    "id": "c39338e4-93fc-4fcb-8de5-bbbd2860d950"
                },
                "virtualInterface": [
                    {
                        "name": "Vlan1",
                        "id": "631365df-5b91-437f-9b54-1299b8316cb4"
                    }
                ]
            },
            "egressInterface": {
                "physicalInterface": {
                    "name": "TenGigabitEthernet1/1/1",
                    "id": "a385c26c-8d6c-4efa-9cac-1a2febddc789"
                }
            },
            "id": "74

This result can be pretty printed

In [7]:
for device in pathTrace.response.networkElementsInfo:
    print("{name}: {ip}\n".format(name=device.name, ip=device.ip))

None: 10.10.22.114

cat_9k_2.abc.inc: 10.10.22.70

cs3850.abc.inc: 10.10.22.69

cat_9k_1.abc.inc: 10.10.22.66

None: 10.10.22.98



<hr>
<font color=blue>
<h2> Challenge</h2>
Change the call to request device statistics and display those.
<p>
<i><b> Hint: "inclusions" : ["DEVICE-STATS"] needs to be added to the payload body</b></i>
<p>
<b> Bonus, also display the version of code for the device
</font>
<hr>